In [ ]:
# 3. SUPPORT VECTOR REGRESSION (SVR)
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import pickle


In [5]:
X_train = pd.read_csv("../data/processed/X_train.csv")
y_train = pd.read_csv("../data/processed/y_train.csv")["RUL_steps"].values
X_val = pd.read_csv("../data/processed/X_val.csv")
y_val = pd.read_csv("../data/processed/y_val.csv")["RUL_steps"].values
train_balanced =  pd.read_csv("../data/processed/train_balanced.csv")
train_df = pd.read_csv("../data/processed/train_df.csv")
val_df = pd.read_csv("../data/processed/val_df.csv")

In [ ]:
weights = train_balanced['risk_bin'].map({
    'MUY_CRITICO': 8.0,
    'CRITICO': 5.0, 
    'ALTO_RIESGO': 2.0,
    'BAJO_RIESGO': 1.0
}).values



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

svr_model = SVR(kernel='rbf', C=10, epsilon=0.1, gamma='scale')
svr_model.fit(X_train_scaled, y_train, sample_weight=weights)  # ✅ Directo
svr_pred = svr_model.predict(X_val_scaled)

# MÉTRICAS SVR
mask_critica = y_val < 50
svr_mae_total = mean_absolute_error(y_val, svr_pred)
svr_mae_criticos = mean_absolute_error(y_val[mask_critica], svr_pred[mask_critica])
svr_r2_total = r2_score(y_val, svr_pred)

print(f"📊 SVR:")
print(f"MAE total:     {svr_mae_total:.2f}")
print(f"MAE RUL<50:    {svr_mae_criticos:.2f}")
print(f"R² total:      {svr_r2_total:.4f}")
print(f"Nº críticos:   {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")
print()




In [ ]:
with open("../../data/svr.pkl", "wb") as f:
    pickle.dump(svr_model, f)